In [117]:
import tensorflow as tf
import os
import tfx
from tfx.components import ImportExampleGen , CsvExampleGen
from tfx.orchestration import metadata
from tfx.proto import example_gen_pb2, trainer_pb2
import tensorflow as tf
print(tf.__version__)
import tfx
print(tfx.__version__)



2.13.1
1.14.0


In [118]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

The tfx.orchestration.experimental.interactive.notebook_extensions.skip extension is already loaded. To reload it, use:
  %reload_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip


In [119]:
base_dir = os.getcwd()
Tf_record_data_dir = os.path.join(os.pardir, "tfrecord_data")
tf_record_location = os.path.join(base_dir, Tf_record_data_dir)
tf_record_location

'C:\\Users\\ASUS\\building-machine-learning-pipelines\\Untitled Folder\\..\\tfrecord_data'

In [120]:
from tfx.components import CsvExampleGen
from tfx.proto import example_gen_pb2

output = example_gen_pb2.Output(
split_config=example_gen_pb2.SplitConfig(splits=[
example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=6),
example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=2),
example_gen_pb2.SplitConfig.Split(name='test', hash_buckets=2)]))
print(output)

split_config {
  splits {
    name: "train"
    hash_buckets: 6
  }
  splits {
    name: "eval"
    hash_buckets: 2
  }
  splits {
    name: "test"
    hash_buckets: 2
  }
}



In [121]:
example_gen = ImportExampleGen(input_base=tf_record_location , output_config=output)#ImportExampleGen because we are impoting Tfrecord file format

In [122]:
context = InteractiveContext(pipeline_root='../pipeline_root_learning/TFX_components_artifects_store')   # ".." means the parent directory
context.run(example_gen)

ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 13
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=ImportExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [123]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 14
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [124]:
from tfx.components import SchemaGen 
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 15
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [125]:
_housing_transform_module_file = 'housing_transform.py'
# Ignore TF warning messages
tf.get_logger().setLevel('ERROR')

# Instantiate the Transform component
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'], # tfExample
    schema=schema_gen.outputs['schema'],# Schema
    module_file=os.path.abspath(_housing_transform_module_file)) # preprocessing_fn()
# Run the component
context.run(transform)


ExecutionResult(
    component_id: Transform
    execution_id: 16
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [175]:
_tuner_module_file = 'tuner.py'
# Setup the Tuner component  /// https://www.tensorflow.org/tfx/api_docs/python/tfx/v1/components/Tuner
tuner = tfx.components.Tuner(
    module_file=_tuner_module_file,
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=512),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=100)
    )
context.run(tuner, enable_cache=False)

Trial 30 Complete [00h 00m 59s]
val_loss: 0.2647366225719452

Best val_loss So Far: 0.24197286367416382
Total elapsed time: 00h 16m 30s
Results summary
Results in ../pipeline_root_learning/TFX_components_artifects_store\.temp\23\housing_tuning
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 0016 summary
Hyperparameters:
units_1: 430
units_2: 125
units_3: 300
units_4: 175
learning_rate: 0.001
tuner/epochs: 15
tuner/initial_epoch: 5
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0012
Score: 0.24197286367416382

Trial 0024 summary
Hyperparameters:
units_1: 235
units_2: 275
units_3: 300
units_4: 150
learning_rate: 0.001
tuner/epochs: 15
tuner/initial_epoch: 5
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0022
Score: 0.26136788725852966

Trial 0029 summary
Hyperparameters:
units_1: 175
units_2: 225
units_3: 225
units_4: 125
learning_rate: 0.001
tuner/epochs: 15
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.2647366225719452

Trial 0025 summary
H

TypeError: '<' not supported between instances of 'int' and 'NoneType'

In [176]:
_trainer_module_file = 'trainer.py'
# Setup the Trainer component
trainer = tfx.components.Trainer(
    module_file=_trainer_module_file,
    examples=transform.outputs['transformed_examples'],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval']))
context.run(trainer, enable_cache=False)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 longitude_xf (InputLayer)   [(None, 1)]                  0         []                            
                                                                                                  
 latitude_xf (InputLayer)    [(None, 1)]                  0         []                            
                                                                                                  
 total_rooms_xf (InputLayer  [(None, 1)]                  0         []                            
 )                                                                                                
                                                                                                  
 total_bedrooms_xf (InputLa  [(None, 1)]                  0         []                      

ExecutionResult(
    component_id: Trainer
    execution_id: 24
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

#### Path of TRAINED MODEL

In [129]:
# # Get artifact uri of trainer model output
model_artifact_dir = trainer.outputs['model'].get()[0].uri
model_dir = os.path.join(model_artifact_dir, 'Format-Serving')
model_dir
# model_dir = "my_housing_model/0003"

'../pipeline_root_learning/TFX_components_artifects_store\\Trainer\\model\\18\\Format-Serving'

In [134]:
model = tf.saved_model.load(export_dir=model_dir)
predict_fn = model.signatures["serving_default"]
predict_fn 

<ConcreteFunction signature_wrapper_serve_tf_examples_fn(*, examples) at 0x20D17586850>

In [159]:
def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))

# Prepare your input data (assuming you have a single data point for prediction)
input_data = {
    "longitude": 52,
    "latitude": 1111110,
    "housing_median_age": 40,
    "total_rooms": 111110,
    "total_bedrooms":52,
    "population": 454440,
    "households": 4554540,
    "median_income": 54578,
    "median_house_value":5
}

# Convert input data to tf.train.Example format
example = tf.train.Example(features=tf.train.Features(feature={
    "longitude": _float_feature(input_data["longitude"]),
    "latitude": _float_feature(input_data["latitude"]),
    "housing_median_age": _float_feature(input_data["housing_median_age"]),
    "total_rooms": _float_feature(input_data["total_rooms"]),
    "total_bedrooms": _float_feature(input_data["total_bedrooms"]),
    "population": _float_feature(input_data["population"]),
    "households": _float_feature(input_data["households"]),
    "median_income": _float_feature(input_data["median_income"]),
    "median_house_value": _float_feature(input_data["median_house_value"])
}))

# Serialize the example
serialized_example = example.SerializeToString()

In [160]:
print(predict_fn(tf.constant([serialized_example])))

{'outputs': <tf.Tensor: shape=(1, 5), dtype=float32, numpy=array([[0., 0., 1., 0., 0.]], dtype=float32)>}


In [130]:
path_to_savedmodel=model_dir

In [131]:
!saved_model_cli show --dir {path_to_savedmodel} --tag_set serve

The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "__saved_model_init_op"
SignatureDef key: "serving_default"
SignatureDef key: "transform_features"


In [55]:
!saved_model_cli show --dir {path_to_savedmodel} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['households_xf'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: serving_default_households_xf:0
    inputs['housing_median_age_xf'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: serving_default_housing_median_age_xf:0
    inputs['latitude_xf'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: serving_default_latitude_xf:0
    inputs['longitude_xf'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)

2024-05-26 16:53:41.734691: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE SSE2 SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [25]:
## serving

# i have saved the model with two different approch in trainer script and utilizing the second approch's saved madel loaction for tensorflow serving
!docker run -p 8500:8500 \
-p 8501:8501 \
--mount type=bind,source="C:\Users\ASUS\building-machine-learning-pipelines\Untitled Folder\my_housing_model",target=/models/my_model \
-e MODEL_NAME=my_model -t tensorflow/serving 
#source path should be absolute means from root directory i.e
#path should not contain model version 

^C


### Path of Evaluation Set

In [140]:
# eval_dir = "../pipeline_root_learning/TFX_components_artifects_store/Transform/transformed_examples/44/Split-eval/transformed_examples-00000-of-00001.gz"
eval_dir = "../pipeline_root_learning/TFX_components_artifects_store/ImportExampleGen/examples/13/Split-eval/data_tfrecord-00000-of-00001.gz"

# TFMA (Tensorflow model Analysis)

In [141]:
import tensorflow_model_analysis as tfma
import tensorflow as tf

import sys
import os

# stop tf warnings 
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [44]:
# may need to run this every time
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter nbextension install --py --symlink tensorflow_model_analysis --sys-prefix
!jupyter nbextension enable --py tensorflow_model_analysis --sys-prefix
# then refresh browser page

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Installing d:\Users\ASUS\anaconda3\envs\environment_tfx\lib\site-packages\tensorflow_model_analysis\static -> tensorflow_model_analysis
- Validating: ok

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable tensorflow_model_analysis --py --sys-prefix
    
Enabling notebook extension tensorflow_model_analysis/extension...
      - Validating: ok


### It takes a saved model and an evaluation dataset (tfrecord format) as input

In [142]:
## vizualising the evaluation spit data produced by transform component after applying all preprocessing steps

dataset = tf.data.TFRecordDataset(eval_dir, compression_type="GZIP")
from google.protobuf.json_format import MessageToDict

# Define a helper function to get individual examples
def get_records(dataset, num_records):
    '''Extracts records from the given dataset.
    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    '''
    
    # initialize an empty list
    records = []
    
    # Use the `take()` method to specify how many records to get
    for tfrecord in dataset.take(num_records):
        
        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()
        
        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()
        
        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)
        
        # convert the protocol bufffer message to a Python dictionary
        example_dict = (MessageToDict(example))
        
        # append to the records list
        records.append(example_dict)
        
    return records



# Get 3 records from the dataset
from pprint import pprint 

sample_records = get_records(dataset,5)

# Print the output
pprint(sample_records)

[{'features': {'feature': {'households': {'floatList': {'value': [4290.0]}},
                           'housing_median_age': {'floatList': {'value': [21.0]}},
                           'latitude': {'floatList': {'value': [34.8487]}},
                           'longitude': {'floatList': {'value': [-116.12929]}},
                           'median_house_value': {'floatList': {'value': [216737.3]}},
                           'median_income': {'floatList': {'value': [3.4760513]}},
                           'ocean_proximity': {'bytesList': {'value': ['TkVBUiBPQ0VBTg==']}},
                           'population': {'floatList': {'value': [1220.0]}},
                           'total_bedrooms': {'floatList': {'value': [2938.0]}},
                           'total_rooms': {'floatList': {'value': [31901.0]}}}}},
 {'features': {'feature': {'households': {'floatList': {'value': [1722.0]}},
                           'housing_median_age': {'floatList': {'value': [19.0]}},
                    

In [143]:
_EVAL_DATA_FILE = eval_dir # path to evaluation data
_MODEL_DIR = model_dir#"my_housing_model/0004"       #model_dir # path to my model directory


In [144]:
print(_EVAL_DATA_FILE)
print(_MODEL_DIR)

../pipeline_root_learning/TFX_components_artifects_store/ImportExampleGen/examples/13/Split-eval/data_tfrecord-00000-of-00001.gz
../pipeline_root_learning/TFX_components_artifects_store\Trainer\model\18\Format-Serving


In [145]:
## First, the SavedModel must be converted to an EvalSharedModel:

eval_shared_model = tfma.default_eval_shared_model(
    eval_saved_model_path=_MODEL_DIR, tags=[tf.saved_model.SERVING])

In [161]:
##  tfma.EvalConfig object must be defined that provides specifications for the model, metrics, and slices that are to be evaluated
eval_config=tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(signature_name='serving_default',
                                    label_key='ocean_proximity_xf' , 
                                    preprocessing_function_names=['transform_features'])], # model specs
        slicing_specs=[tfma.SlicingSpec(), tfma.SlicingSpec(feature_keys=['housing_median_age'])], # slicing_specs is a list of tfma.SlicingSpec objects. Each SlicingSpec specifies a way to slice the data.
        metrics_specs=[
              tfma.MetricsSpec(metrics=[
                tfma.MetricConfig(class_name='CategoricalAccuracy'),
                tfma.MetricConfig(class_name='Precision'),
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='Recall'),
                tfma.MetricConfig(class_name='AUC'),
                tfma.MetricConfig(class_name='F1Score')
              ])])



print(eval_config)


# All the metrics (based on the type of model) can be
# displayed in TFMA by providing them in the metrics_specs argument to the
# EvalConfig:

model_specs {
  signature_name: "serving_default"
  label_key: "ocean_proximity_xf"
  preprocessing_function_names: "transform_features"
}
slicing_specs {
}
slicing_specs {
  feature_keys: "housing_median_age"
}
metrics_specs {
  metrics {
    class_name: "CategoricalAccuracy"
  }
  metrics {
    class_name: "Precision"
  }
  metrics {
    class_name: "ExampleCount"
  }
  metrics {
    class_name: "Recall"
  }
  metrics {
    class_name: "AUC"
  }
  metrics {
    class_name: "F1Score"
  }
}



In [162]:
slices = [tfma.slicer.SingleSliceSpec(),
          tfma.slicer.SingleSliceSpec(columns=['housing_median_age'])]

In [163]:
slices

[SingleSliceSpec(columns=frozenset(), features=frozenset()),
 SingleSliceSpec(columns=frozenset({'housing_median_age'}), features=frozenset())]

In [164]:
# eval_result = tfma.run_model_analysis(
#     eval_shared_model=eval_shared_model,
#     eval_config=eval_config,
#     data_location=eval_dir,
#     output_path="./eval_result_DO_0.2_",
#     file_format='tfrecords',
#     slice_spec = slices)

#Then, run the model analysis step:
eval_result = tfma.run_model_analysis(
    eval_shared_model=eval_shared_model,
    eval_config=eval_config,
    data_location=_EVAL_DATA_FILE,
    output_path="./eval_result_DO_0.1_another", 
    file_format='tfrecords',
    slice_spec = slices)

In [165]:
tfma.view.render_slicing_metrics(eval_result)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…

### COMPARING 2 MODELS

We can also use TFMA to compare our metrics across multiple models. For example,
these may be the same model trained on different datasets, or two models with different
hyperparameters trained on the same dataset

In [17]:
_MODEL_DIR_2 = "my_housing_model/0004"

eval_shared_model_2 = tfma.default_eval_shared_model(
    eval_saved_model_path=_MODEL_DIR_2, tags=[tf.saved_model.SERVING])

eval_result_2 = tfma.run_model_analysis(
    eval_shared_model=eval_shared_model_2,
    eval_config=eval_config,
    data_location=_EVAL_DATA_FILE,
    output_path="./eval_result_DO_0.1",
    file_format='tfrecords',
    slice_spec = slices)

In [23]:
tfma.view.render_slicing_metrics(eval_result_2 , slicing_spec=slices[0])

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…

In [68]:
eval_results_from_disk = tfma.load_eval_results(
    ['./eval_result_DO_0.2','./eval_result_DO_0.1'])

In [69]:
# bug - only works reliably in Colab
tfma.view.render_time_series(eval_results_from_disk, slices[0])


TimeSeriesViewer(config={'isModelCentric': True}, data=[{'metrics': {'': {'': {'categorical_accuracy': {'doubl…

## Model Analysis for Fairness

In [25]:
# ! pip install tensorboard_plugin_fairness_indicators

**Next, we use TFMA to evaluate the model and ask it to calculate metrics for a set of
decision thresholds we supply. This is supplied to TFMA in the metrics_spec argument
for the EvalConfig, along with any other metrics we wish to calculate:**

In [35]:
##  tfma.EvalConfig object must be defined that provides specifications for the model, metrics, and slices that are to be evaluated
eval_config=tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key='ocean_proximity_xf')], # model specs
        slicing_specs=[tfma.SlicingSpec(), tfma.SlicingSpec(feature_keys=['housing_median_age'])], # slicing_specs is a list of tfma.SlicingSpec objects. Each SlicingSpec specifies a way to slice the data.
        metrics_specs=[
              tfma.MetricsSpec(metrics=[
                tfma.MetricConfig(class_name='CategoricalAccuracy'),
                tfma.MetricConfig(class_name='Precision'),
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='Recall'),
                tfma.MetricConfig(class_name='AUC'),
                tfma.MetricConfig(class_name='F1Score'),
                tfma.MetricConfig(class_name='FairnessIndicators',
                              config='{"thresholds":[0.25, 0.5, 0.75]}')
              ])])


In [36]:
##Then run the model analysis step as before via tfma.run_model_analysis.
eval_result_2 = tfma.run_model_analysis(
    eval_shared_model=eval_shared_model_2,
    eval_config=eval_config,
    data_location=_EVAL_DATA_FILE,
    output_path="./eval_result_fairness",
    file_format='tfrecords',
    slice_spec = slices)

In [37]:
from tensorboard_plugin_fairness_indicators import summary_v2

writer = tf.summary.create_file_writer('./fairness_indicator_logs') ## write the TFMA evaluation result to a log directory so that it can be picked up
by TensorBoard:
with writer.as_default():
    summary_v2.FairnessIndicators('./eval_result_fairness', step=1)
    writer.close()

In [39]:
%load_ext tensorboard
%tensorboard --logdir=./fairness_indicator_logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 9924), started 0:09:17 ago. (Use '!kill 9924' to kill it.)

## WHAT-IF TOOLS

In [43]:
# !pip install witwidget

In [66]:
# !pip uninstall protobuf
# !pip install --no-binary protobuf protobuf
# # !pip show protobuf # previous version of protobuf 3.20.3


^C


In [69]:
# # from witwidget.notebook.visualization import WitConfigBuilder
# from witwidget.notebook.visualization import WitWidget

In [52]:
## vizualising the evaluation spit data produced by transform component after applying all preprocessing steps

dataset = tf.data.TFRecordDataset(eval_dir, compression_type="GZIP")
from google.protobuf.json_format import MessageToDict

# Define a helper function to get individual examples
def get_records(dataset, num_records):
    '''Extracts records from the given dataset.
    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    '''
    
    # initialize an empty list
    records = []
    
    # Use the `take()` method to specify how many records to get
    for tfrecord in dataset.take(num_records):
        
        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()
        
        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()
        
        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)
        
        # convert the protocol bufffer message to a Python dictionary
        example_dict = (MessageToDict(example))
        
        # append to the records list
        records.append(example_dict)
        
    return records



# Get 3 records from the dataset
from pprint import pprint 

sample_records = get_records(dataset,1000)

# Print the output
# pprint(sample_records)

In [54]:
# sample_records

In [55]:
model = tf.saved_model.load(export_dir=_MODEL_DIR)
predict_fn = model.signatures['serving_default']

In [57]:
def predict(examples):
    preds = model.signatures['serving_default'](examples=tf.constant([example.SerializeToString() for example in examples]))
    return preds['outputs'].numpy()

# Analysis and Validation in TFX

## ResolverNode

In [53]:
# !pip uninstall shapely -y  # after install tfx remove shapley

In [166]:
# Setup the Resolver node to find the latest blessed model
from tfx import v1 as tfx

model_resolver = tfx.dsl.Resolver(
      strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.dsl.Channel(
          type=tfx.types.standard_artifacts.ModelBlessing)).with_id(
              'latest_blessed_model_resolver')

# Run the Resolver node
context.run(model_resolver)

ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 19
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## EVALUATOR COMPONENT

In [169]:
# ## first define eval config with some thresholds
eval_config_threshold = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(preprocessing_function_names=['transform_features'],label_key='ocean_proximity_xf')],
    slicing_specs=[
        tfma.SlicingSpec(),
        tfma.SlicingSpec(feature_keys=['housing_median_age'])
    ],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(
                class_name='CategoricalAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(lower_bound={'value': 0.44}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value': 0.01}
                    )
                )
            )
        ])
    ]
)

# In this eval_config_threshold, we state that the CategoricalAccuracy must be over 0.44, and we want the model to
# be validated if its CategoricalAccuracy is at least 0.01 higher than that of the baseline model. Any
# other metric can be added in place of CategoricalAccuracy, but note that the metric you add must
# also be included in the list of in the MetricsSpec. metrics

In [170]:
from tfx.components import Evaluator
evaluator = Evaluator(examples=example_gen.outputs['examples'], model=trainer.outputs['model'],
baseline_model=model_resolver.outputs['model'],
eval_config=eval_config_threshold)
context.run(evaluator)

ExecutionResult(
    component_id: Evaluator
    execution_id: 21
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [171]:
context.show(evaluator.outputs['evaluation'])


SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…

*Compares the new model's performance to a baseline (if available). On the first run, there is no previous model, so no comparison is needed.
Outputs a blessing if the model meets performance criteria.*

A "blessed" model is a model that has been evaluated and deemed to meet or exceed predefined performance criteria. This model is considered good enough for production use and can be deployed to a serving location. The process involves the following steps:

A "baseline" model is an existing model that serves as a reference point for comparison when evaluating new models. The baseline model is usually the last blessed model or an earlier version of the model that has been successfully used in production.

## Validation in the Evaluator Component

In [173]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
print(tfma.load_validation_result(eval_result))
#If the validation check passes, it will return the following result:
#validation_ok: true

metric_validations_per_slice {
  slice_key {
    single_slice_keys {
      column: "housing_median_age"
      float_value: 5.0
    }
  }
  failures {
    metric_key {
      name: "categorical_accuracy"
      example_weighted {
      }
    }
    metric_threshold {
      value_threshold {
        lower_bound {
          value: 0.44
        }
      }
    }
    metric_value {
      double_value {
        value: 0.3965517282485962
      }
    }
  }
}
metric_validations_per_slice {
  slice_key {
    single_slice_keys {
      column: "housing_median_age"
      float_value: 13.0
    }
  }
  failures {
    metric_key {
      name: "categorical_accuracy"
      example_weighted {
      }
    }
    metric_threshold {
      value_threshold {
        lower_bound {
          value: 0.44
        }
      }
    }
    metric_value {
      double_value {
        value: 0.41333332657814026
      }
    }
  }
}
metric_validations_per_slice {
  slice_key {
    single_slice_keys {
      column: "housing_median

**the above defines that their is no more imporvement in model hence model will not be pushed**

In [102]:
##We can also show the TFMA visualization with:
tfma_result = tfma.load_eval_result(eval_result)
##And we can load Fairness Indicators with:
tfma.addons.fairness.view.widget_view.render_fairness_indicator(tfma_result)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'categorical…

## TFX Pusher Component

In [174]:
from tfx.components import Pusher
from tfx.proto import pusher_pb2
_serving_model_dir = "serving_model_dir"
pusher = Pusher(model=trainer.outputs['model'],
                model_blessing=evaluator.outputs['blessing'],
                push_destination=pusher_pb2.PushDestination(filesystem=pusher_pb2.PushDestination.Filesystem(base_directory=_serving_model_dir)))
context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 22
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

Once the new model has been pushed to the serving directory, it can then be picked
up by TensorFlow Serving